# Imports

In [ ]:
from PIL import Image
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import zipfile
from subprocess import check_output
import matplotlib.pyplot as plt
import matplotlib
import time
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import seaborn as sns
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from random import sample 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from scipy import ndimage
import copy
import warnings
from multiprocessing import Pool

#  Load Images from dataset
Check the readme file to download the dataset

In [ ]:
in_imgs_path = [] # Saves the path of images
in_imgs = [] # Saves the name of images

for dirname, _, filenames in os.walk("./data/train_data"):
    for filename in filenames:
        in_imgs_path.append(os.path.join(dirname, filename))
        in_imgs.append(filename[:-5])

In [ ]:
len(in_imgs_path)

In [ ]:
# Drop unuseful columns from csv

new_lbls_0 = pd.read_csv("./data/df0.csv")
new_lbls_0 = new_lbls_0.drop(columns=['path', 'level_cat','exists','PatientId','Unnamed: 0'])

new_lbls_1 = pd.read_csv("./data/df1.csv")
new_lbls_1 = new_lbls_1.drop(columns=['path', 'level_cat','exists','PatientId','Unnamed: 0'])

new_lbls_2 = pd.read_csv("./data/df2.csv")
new_lbls_2 = new_lbls_2.drop(columns=['path', 'level_cat','exists','PatientId','Unnamed: 0'])

new_lbls_3 = pd.read_csv("./data/df3.csv")
new_lbls_3 = new_lbls_3.drop(columns=['path', 'level_cat','exists','PatientId','Unnamed: 0'])

new_lbls_4 = pd.read_csv("./data/df4.csv")
new_lbls_4 = new_lbls_4.drop(columns=['path', 'level_cat','exists','PatientId','Unnamed: 0'])

# Aggregate Dataset into a Dataframe

In [ ]:
in_lbls = pd.concat([new_lbls_0, new_lbls_1, new_lbls_2, new_lbls_3, new_lbls_4], ignore_index=True)
display(in_lbls)

In [ ]:
left_lbls = in_lbls[in_lbls["eye"] == 1]
display(left_lbls)

In [ ]:
right_lbls = in_lbls[in_lbls["eye"] == 0]
display(right_lbls)

In [ ]:
sns.countplot("eye",data= in_lbls, color="black")

# Visualize label distribution

In [ ]:
sns.countplot("level",data= left_lbls, color="black")

# Data Preprocessing

*Trimming*:  

    Converts image to grayscale using cv2, then computes binary matrix
    of the pixels that are above a certain threshold, then takes out
    the first row where a certain percetage of the pixels are above the
    threshold will be the first clip point. Same idea for col, max row, max col.

*resize_maintain_aspect*:   

    Resizes the image by padding black pixels to preserve aspect ratio

In [ ]:
def trim(im):

    percentage = 0.02

    img = np.array(im)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    im = img_gray > 0.1 * np.mean(img_gray[img_gray != 0])
    row_sums = np.sum(im, axis=1)
    col_sums = np.sum(im, axis=0)
    rows = np.where(row_sums > img.shape[1] * percentage)[0]
    cols = np.where(col_sums > img.shape[0] * percentage)[0]
    min_row, min_col = np.min(rows), np.min(cols)
    max_row, max_col = np.max(rows), np.max(cols)
    im_crop = img[min_row : max_row + 1, min_col : max_col + 1]
    return Image.fromarray(im_crop)


def resize_maintain_aspect(image, desired_size):

    old_size = image.size  # old_size[0] is in (width, height) format
    ratio = float(desired_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    im = image.resize(new_size, Image.ANTIALIAS)
    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size - new_size[0]) // 2, (desired_size - new_size[1]) // 2))
    return new_im

# Used in previous version of implementation but deprecated now
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img
    
# Used in previous version of implementation but deprecated now
def load_ben_color(img, sigmaX=5):
    image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (img_width, img_height))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

# Raw Dataset Preparation

Specify the number of samples per class in the variable named *trgt_nbr_samples_per_class*.  
Then specify the target *img_width* and *img_height* to resize the images.  
N.B: Increasing the image size affects positively on getting a higher accuracy & Kappa Score

In [ ]:
trgt_nbr_samples_per_class = 10000

# Resize params
img_width = int(100)
img_height = int(100)

imgs_left_flat = [] # Resulting dataset for only left images
imgs_left_label = [] # Label of left eye images

imgs_right_flat = [] # Resulting dataset for only right images
imgs_right_label = [] # Label of right eye images


# Local variables to check that the number of samples
# we're using for our dataset doesn't exceed the trgt_nbr_samples_per_class
class_samples = {}
class_samples[0] = 0
class_samples[1] = 0
class_samples[2] = 0
class_samples[3] = 0
class_samples[4] = 0

for image in tqdm(in_imgs_path):
    try:
        base_folder = os.path.basename(image)
        fileName = os.path.splitext(base_folder)[0]
        
        patient_name = fileName

        patient = patient_name.split("_",1)[0]
        eye = patient_name.split("_",1)[1]

        tmp_lbl = in_lbls.loc[in_lbls.image==fileName, 'level'].values[0] # reference is variable called eye
        
        
        if class_samples[tmp_lbl] < trgt_nbr_samples_per_class:
            class_samples[tmp_lbl] = class_samples[tmp_lbl] + 1
        else:
            continue

        if(eye == "left"):
            # Get the left eye and then its associated right eye for the same patient.
            im = Image.open(image) 
            im = trim(im)
            im = resize_maintain_aspect(im, desired_size=img_width)
            img = im.convert('L')
            
            imgs_left_label.append(tmp_lbl)
            imgs_left_flat.append(np.array(img).flatten())
            
            # Right eye of same patient
            im = Image.open(image.replace("left","right")) 
            im = trim(im)
            im = resize_maintain_aspect(im, desired_size=img_width)
            img = im.convert('L')

            imgs_right_label.append(tmp_lbl)
            imgs_right_flat.append(np.array(img).flatten())
        else:
            # Get the right eye and then its associated left eye for the same patient.
            im = Image.open(image) 
            im = trim(im)
            im = resize_maintain_aspect(im, desired_size=img_width)
            img = im.convert('L')

            imgs_right_label.append(tmp_lbl)
            imgs_right_flat.append(np.array(img).flatten())
            
            # Left eye of same patient
            im = Image.open(image.replace("right","left")) 
            im = trim(im)
            im = resize_maintain_aspect(im, desired_size=img_width)
            img = im.convert('L')
            

            imgs_left_label.append(tmp_lbl)
            imgs_left_flat.append(np.array(img).flatten())
        
        
      
    except:
        continue;
        #print("Exception")

In [ ]:
lvls_bincount = np.bincount(imgs_left_label)
print(lvls_bincount)

In [ ]:
lvls_bincount = np.bincount(imgs_right_label)
print(lvls_bincount)

In [ ]:
print(len(imgs_left_label))
print(len(imgs_left_flat))

print(len(imgs_right_label))
print(len(imgs_right_flat))

# Data Augmentation and Dataset Balancing
Augment the data to have equal number of samples for each class

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range = (0.5, 1.2),
    fill_mode='nearest')

In [ ]:
def getRand_Left_ImgFromClass(classIndex):
    values = np.array(imgs_left_label)
    searchval = classIndex
    indeces = np.where(values == searchval)[0]
    index = np.random.choice(indeces, 1, replace=False)
    return index[0]

In [ ]:
def getRand_Right_ImgFromClass(classIndex):
    values = np.array(imgs_right_label)
    searchval = classIndex
    indeces = np.where(values == searchval)[0]
    index = np.random.choice(indeces, 1, replace=False)
    return index[0]

In [ ]:
def generateSamples(x, nbrOfSamples):
    samples = []
    i = 0
    for batch in datagen.flow(x, batch_size=1):
        augImage = batch[0]
        augImage = augImage.astype('float32')
        augImage /= 255
        samples.append(augImage.flatten())
        i += 1
        if i > (nbrOfSamples - 1):
            return samples 

In [ ]:
# Test to check a resulting image sample for right eye
i = 0
indx = getRand_Right_ImgFromClass(i)
img = imgs_right_flat[indx]
img = img.reshape((img_width,img_width,1))
plt.imshow(img)

In [ ]:
# Test to check a resulting image sample for left eye
i = 0
indx = getRand_Left_ImgFromClass(i)
img = imgs_left_flat[indx]
img = img.reshape((img_width,img_width,1))
plt.imshow(img)

## Aggregating all previous parts together to generate new samples for left and right eye

In [ ]:
# Left eye
print("Generating Left Images")
nbSamples = 5 # Specifies number of new augmented data samples to be generated given one "good" initial sample
lvls_bincount = np.bincount(imgs_left_label)
for classIndex,count in enumerate(lvls_bincount):
    if count < trgt_nbr_samples_per_class:
        print(classIndex)
        nb_imgs_to_gnrt = trgt_nbr_samples_per_class - count
        for i in range(int(nb_imgs_to_gnrt/nbSamples)):
            # Get rand image for this classIndex
            indx = getRand_Left_ImgFromClass(classIndex)
            img = imgs_left_flat[indx]
            img = img.reshape((img_width,img_width,1))
            x = img_to_array(img)  # this is a Numpy array with shape (3, img_height, img_width)
            x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, img_height, img_width)
            samples = generateSamples(x,nbSamples)
            for j in range(nbSamples):
                imgs_left_flat.append(samples[j])
                imgs_left_label.append(classIndex)

# Right eye
print("Generating Right Images")
nbSamples = 5 # Specifies number of new augmented data samples to be generated given one "good" initial sample
lvls_bincount = np.bincount(imgs_right_label)
for classIndex,count in enumerate(lvls_bincount):
    if count < trgt_nbr_samples_per_class:
        print(classIndex)
        nb_imgs_to_gnrt = trgt_nbr_samples_per_class - count
        for i in range(int(nb_imgs_to_gnrt/nbSamples)):
            # Get rand image for this classIndex
            indx = getRand_Right_ImgFromClass(classIndex)
            img = imgs_right_flat[indx]
            img = img.reshape((img_width,img_width,1))
            x = img_to_array(img)  # this is a Numpy array with shape (3, img_height, img_width)
            x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, img_height, img_width)
            samples = generateSamples(x,nbSamples)
            for j in range(nbSamples):
                imgs_right_flat.append(samples[j])
                imgs_right_label.append(classIndex)

# Dataset split for Left and Right

In [ ]:
# Dataset of Left Images

imgs_left_flat = np.asarray(imgs_left_flat)
imgs_left_label = np.asarray(imgs_left_label)

X_left = imgs_left_flat
y_left = imgs_left_label

X_train_left, X_test_left, y_train_left, y_test_left = train_test_split(X_left, y_left, test_size=0.3, random_state=5)

X_train_left = X_train_left.reshape(X_train_left.shape[0], img_width, img_height,1) # add ,3 if RGB
X_test_left = X_test_left.reshape(X_test_left.shape[0], img_width, img_height,1)  # add ,3 if RGB


X_train_left = X_train_left.astype('float32')
X_test_left = X_test_left.astype('float32')

X_train_left /= 255
X_test_left /= 255


print(X_train_left.shape)
print(y_train_left.shape)
print(X_test_left.shape)
print(y_test_left.shape)

In [ ]:
# Dataset of Right Images

imgs_right_flat = np.asarray(imgs_right_flat)
imgs_right_label = np.asarray(imgs_right_label)

X_right = imgs_right_flat
y_right = imgs_right_label

X_train_right, X_test_right, y_train_right, y_test_right = train_test_split(X_right, y_right, test_size=0.9998, random_state=5)

X_train_right = X_train_right.reshape(X_train_right.shape[0], img_width, img_height,1) # add ,3 if RGB
X_test_right = X_test_right.reshape(X_test_right.shape[0], img_width, img_height,1)  # add ,3 if RGB

X_train_right = X_train_right.astype('float32')
X_test_right = X_test_right.astype('float32')

X_train_right /= 255
X_test_right /= 255


print(X_train_right.shape)
print(y_train_right.shape)
print(X_test_right.shape)
print(y_test_right.shape)

In [ ]:
batch_size = 20
nb_classes = 5
nb_epoch = 10

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_left = to_categorical(y_train_left, nb_classes)
y_test_left = to_categorical(y_test_left, nb_classes)

y_train_right = to_categorical(y_train_right, nb_classes)
y_test_right = to_categorical(y_test_right, nb_classes)

In [ ]:
c = [np.argmax(row) for row in y_train_left]

In [ ]:
training_bincount = np.bincount(c)
print(training_bincount)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax_labels = [0,1,2,3,4]
ax_data = training_bincount
ax.bar(ax_labels,ax_data)
plt.show()

# Quadratic Kappa Score function

In [ ]:
from sklearn.metrics import confusion_matrix

def quadratic_kappa(actuals, preds, N=5):
    """This function calculates the Quadratic Kappa Metric used for Evaluation at Kaggle.
    It returns the Quadratic Weighted Kappa metric score between the actual and the predicted values."""
    w = np.zeros((N,N))
    O = confusion_matrix(actuals, preds)
    for i in range(len(w)): 
        for j in range(len(w)):
            w[i][j] = float(((i-j)**2)/(N-1)**2)
    
    act_hist=np.zeros([N])
    for item in actuals: 
        act_hist[item]+=1
    
    pred_hist=np.zeros([N])
    for item in preds: 
        pred_hist[item]+=1
                         
    E = np.outer(act_hist, pred_hist);
    E = E/E.sum();
    O = O/O.sum();
    
    num=0
    den=0
    for i in range(len(w)):
        for j in range(len(w)):
            num+=w[i][j]*O[i][j]
            den+=w[i][j]*E[i][j]
    return (1 - (num/den))

# Defining two Bayesian CNNs
One used for left eye images only and another user for right eye images only

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp


# Left eye model

model_in = tf.keras.layers.Input(shape=(img_width,img_height,1))

conv_1 = tfp.python.layers.Convolution2DFlipout(32, kernel_size=(5, 5), padding="same", strides=2)
x = conv_1(model_in)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

conv_2 = tfp.python.layers.Convolution2DFlipout(64, kernel_size=(3, 3), padding="same", strides=2)
x = conv_2(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

conv_3 = tfp.python.layers.Convolution2DFlipout(128, kernel_size=(3, 3), padding="same", strides=2)
x = conv_3(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Flatten()(x)

dense_1 = tfp.python.layers.DenseFlipout(2000, activation='relu')
x = dense_1(x)

dense_2 = tfp.python.layers.DenseFlipout(nb_classes, activation='softmax')
model_out = dense_2(x)  

model = tf.keras.Model(model_in, model_out)

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp

# Right eye model

model_in_r = tf.keras.layers.Input(shape=(img_width,img_height,1))

conv_1_r = tfp.python.layers.Convolution2DFlipout(32, kernel_size=(5, 5), padding="same", strides=2)
x = conv_1_r(model_in_r)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

conv_2_r = tfp.python.layers.Convolution2DFlipout(64, kernel_size=(3, 3), padding="same", strides=2)
x = conv_2_r(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

conv_3_r = tfp.python.layers.Convolution2DFlipout(128, kernel_size=(3, 3), padding="same", strides=2)
x = conv_3_r(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Flatten()(x)

dense_1_r = tfp.python.layers.DenseFlipout(2000, activation='relu')
x = dense_1(x)

dense_2_r = tfp.python.layers.DenseFlipout(nb_classes, activation='softmax')
model_out_r = dense_2_r(x)  

model_right = tf.keras.Model(model_in_r, model_out_r)

## Evidence lower bound loss function

In [ ]:
@tf.function
def elbo_loss(labels, logits):
    loss_en = tf.nn.softmax_cross_entropy_with_logits(labels, logits)
    loss_kl = tf.keras.losses.KLD(labels, logits)
    loss = tf.reduce_mean(tf.add(loss_en, loss_kl))
    return loss

# Training step for right and left models

In [ ]:
@tf.function
def train_step_left(images, labels):
    with tf.GradientTape() as tape:
        logits = model(X_train_left)
        loss = elbo_loss(labels, logits)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

@tf.function
def train_step_right(images, labels):
    with tf.GradientTape() as tape:
        logits = model_right(X_train_right)
        loss = elbo_loss(labels, logits)
    gradients = tape.gradient(loss, model_right.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model_right.trainable_variables))
    return loss

def accuracy(preds, labels):
    return np.mean(np.argmax(preds, axis=1) == np.argmax(labels, axis=1))

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=0.01)

# Main training + evaluation loop

## Left Model

In [ ]:
epochs = 20
times = [] # Stores how much time each training loop took
accs = [] # Stores training accuracy at every epoch
val_accs = [] # Stores validation accuracy at every epoch
losses = [] # loss per epoch
val_losses = [] # val_loss per epoch

for i in range(epochs):
    tic = time.time()
    loss = train_step_left(X_train_left, y_train_left)
    preds = model(X_train_left)
    acc = accuracy(preds, y_train_left)
    accs.append(acc)
    losses.append(loss)
    
    val_preds = model(X_test_left)
    val_loss = elbo_loss(y_test_left, val_preds)
    val_acc = accuracy(y_test_left, val_preds)
    
    val_accs.append(val_acc)
    val_losses.append(val_loss)

    tac = time.time()
    train_time = tac-tic
    times.append(train_time)

    
    print("Epoch: {}: loss = {:7.3f} , accuracy = {:7.3f}, val_loss = {:7.3f}, val_acc={:7.3f} time: {:7.3f}".format(i, loss, acc, val_loss, val_acc, train_time))

In [ ]:
model.summary()

In [ ]:
plt.plot(np.array(accs), label="acc")
plt.plot(np.array(val_accs), label="val_acc")
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

plt.plot(np.array(losses), label="loss")
plt.plot(np.array(val_losses), label="val_loss")
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

from sklearn.metrics import accuracy_score, classification_report

preds = model.predict(X_test_left, batch_size = 32, verbose = 0)
preds_out = np.argmax(preds, -1)
y_test_out = np.argmax(y_test_left, -1)
print('Accuracy on Test Data: %2.2f%%' % (accuracy_score(y_test_out, preds_out)))
print(classification_report(y_test_out, preds_out))

import seaborn as sns
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

def plot_confusion_matrix(labels,preds):
        cm = metrics.confusion_matrix(labels, preds)
        plt.figure(figsize=(6,6))
        plt.imshow(cm, interpolation='nearest', cmap='Blues')
        plt.title('Confusion matrix', size = 15)
        plt.colorbar()
        tick_marks = np.arange(5)
        plt.xticks(tick_marks, ["0", "1", "2", "3","4"], rotation=0, size = 10)
        plt.yticks(tick_marks, ["0", "1", "2", "3","4"], size = 10)
        plt.tight_layout()
        plt.ylabel('Actual label', size = 15)
        plt.xlabel('Predicted label', size = 15)
        width, height = cm.shape 
        for x in range(width):
            for y in range(height):
                plt.annotate(str(cm[x][y]), xy=(y, x), horizontalalignment='center',verticalalignment='center')

plot_confusion_matrix(y_test_out,preds_out)
quadratic_kappa(y_test_out, preds_out)

## Right Model

In [ ]:
epochs = 20
times = [] # Stores how much time each training loop took
accs = [] # Stores training accuracy at every epoch
val_accs = [] # Stores validation accuracy at every epoch
losses = [] # loss per epoch
val_losses = [] # val_loss per epoch


for i in range(epochs):
    tic = time.time()
    loss = train_step_right(X_train_right, y_train_right)
    preds = model_right(X_train_right)
    acc = accuracy(preds, y_train_right)
    accs.append(acc)
    losses.append(loss)
    
    val_preds = model_right(X_test_right)
    val_loss = elbo_loss(y_test_right, val_preds)
    val_acc = accuracy(y_test_right, val_preds)
    
    val_accs.append(val_acc)
    val_losses.append(val_loss)
   
    tac = time.time()
    train_time = tac-tic
    times.append(train_time)
    
    print("Epoch: {}: loss = {:7.3f} , accuracy = {:7.3f}, val_loss = {:7.3f}, val_acc={:7.3f} time: {:7.3f}".format(i, loss, acc, val_loss, val_acc, train_time))

In [ ]:
plt.plot(np.array(accs), label="acc")
plt.plot(np.array(val_accs), label="val_acc")
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

plt.plot(np.array(losses), label="loss")
plt.plot(np.array(val_losses), label="val_loss")
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

from sklearn.metrics import accuracy_score, classification_report

preds = model_right.predict(X_test_right, batch_size = 32, verbose = 0)
preds_out = np.argmax(preds, -1)
y_test_out = np.argmax(y_test_right, -1)
print('Accuracy on Test Data: %2.2f%%' % (accuracy_score(y_test_out, preds_out)))
print(classification_report(y_test_out, preds_out))

import seaborn as sns
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

def plot_confusion_matrix(labels,preds):
        cm = metrics.confusion_matrix(labels, preds)
        plt.figure(figsize=(6,6))
        plt.imshow(cm, interpolation='nearest', cmap='Blues')
        plt.title('Confusion matrix', size = 15)
        plt.colorbar()
        tick_marks = np.arange(5)
        plt.xticks(tick_marks, ["0", "1", "2", "3","4"], rotation=0, size = 10)
        plt.yticks(tick_marks, ["0", "1", "2", "3","4"], size = 10)
        plt.tight_layout()
        plt.ylabel('Actual label', size = 15)
        plt.xlabel('Predicted label', size = 15)
        width, height = cm.shape 
        for x in range(width):
            for y in range(height):
                plt.annotate(str(cm[x][y]), xy=(y, x), horizontalalignment='center',verticalalignment='center')

plot_confusion_matrix(y_test_out,preds_out)
quadratic_kappa(y_test_out, preds_out)

# Saving / Loading models

In [ ]:
# Right model
# model_right.save('./models/right/right_eye2')
# model_right = tf.keras.models.load_model("./models/right/right_eye")

# Left model
# model.save('./models/left/left_eye2')
# model_left = tf.keras.models.load_model("./models/left/left_eye")

# Inference using the two models (taking into consideration the uncertainty)

In [ ]:
x_l = X_left.reshape(X_left.shape[0], img_width, img_height,1) # add ,3 if RGB
x_r = X_right.reshape(X_right.shape[0], img_width, img_height,1) # add ,3 if RGB

y_l = y_left
y_r = y_right

print(X_test_right.shape)
print(x_l[7000:10000].shape)
print(x_r[7000:10000].shape)


print(y_l[7000:10000].shape)
print(y_r[7000:10000].shape)

print(y_r[7000:10000].shape)
accuracy_score(y_l[7000:10000], y_r[7000:10000])

In [ ]:
out_labels = []

# Inference
mc_samples = 5
preds = [model(X_test_left) for _ in range(mc_samples)]
preds = np.concatenate([tf.nn.softmax(y, axis = -1)[:, :, np.newaxis] for y in preds], axis=-1)

preds_right = [model_right(X_test_left) for _ in range(mc_samples)]
preds_right = np.concatenate([tf.nn.softmax(y, axis = -1)[:, :, np.newaxis] for y in preds_right], axis=-1)

In [ ]:
for i in range(len(X_test_left)):
    pred_left = preds[i]
    mean_left = pred_left.mean(axis=1)
    max_left = np.argmax(mean_left)
    
    pred_right = preds_right[i]
    mean_right = pred_right.mean(axis=1)
    max_right = np.argmax(mean_right)
    
    if(max_left > max_right):
        out_labels.append(max_left)
    else:
        out_labels.append(max_right)

In [ ]:
y_out = np.argmax(y_test_left,-1)
y_out

In [ ]:
print("Accuracy: " + str(accuracy_score(y_out, out_labels)))
print("Kappa: " + str(quadratic_kappa(y_out, out_labels)))